In [2]:
import requests
import pymongo

In [9]:
from dotenv import load_dotenv
load_dotenv() 
import os

FINNHUB_KEY = os.environ['FINNHUB_KEY']
FINNHUB_URI = os.environ['FINNHUB_URI']
GG_GEMINI_API = os.environ['GG_GEMINI_API']

1. Get 1 stock from db which there is no name
2. Get basic detail of that stock
3. Generate 10 - 20 random comments for that stock, with random like number
4. Generate random 0 - 8 replies for each comment, with random like number
5. Save comments and likes into db

In [5]:
db_client = pymongo.MongoClient('mongodb://localhost:27017')
collections = db_client['stock_forum']
tbl_stock = collections['tbl_stock']

In [7]:
db_stock = tbl_stock.find_one({'name': None})
print(db_stock)

symbol = ''
if db_stock is not None:
    symbol = db_stock['symbol']

{'_id': ObjectId('678ac0362118fc1550f5d055'), 'symbol': 'TCRRF', 'mic': 'OOTC'}


In [10]:
#query data from Finnhub but limit 30 APIs/second
#https://finnhub.io/docs/api/stock-symbols
def custom_query(get_url):
    print(get_url)
    try:
        r = requests.get(get_url)
        return r.json()
    except Exception as e:
       print(e)
       return r

"country": "CA",
"currency": "CAD",
"estimateCurrency": "CAD",
"exchange": "TSX VENTURE EXCHANGE - NEX",
"finnhubIndustry": "Energy",
"ipo": "2007-04-27",
"logo": "https://static2.finnhub.io/file/publicdatany/finnhubimage/stock_logo/942976380016.png",
"marketCapitalization": 0.309138,
"name": "Terrace Energy Corp",
"phone": "16042827897",
"shareOutstanding": 7.73,
"ticker": "TZR.H.V",
"weburl": "http://terraceenergy.net/",

In [16]:
if symbol != '':
    #1. Get stock detail
    stock_detail_url = FINNHUB_URI + 'stock/profile2?symbol='+symbol+'&token=' + FINNHUB_KEY
    stock_detail = custom_query(stock_detail_url)
    #print(stock_detail)
    if stock_detail is not None and 'name' in stock_detail:
        #update to db
        tbl_stock.update_one({'symbol': symbol}, {'$set':{
                'name': stock_detail['name'],
                'industry': stock_detail['finnhubIndustry'],
                'ipo': stock_detail['ipo'],
                'cap': stock_detail['marketCapitalization'],
                'web': stock_detail['weburl']
            }})


https://finnhub.io/api/v1/stock/profile2?symbol=TCRRF&token=cu5b37pr01qqj8u65ufgcu5b37pr01qqj8u65ug0
